$V(x,y) = \frac{1}{2}\left(x^2+y^2\right) + \left(x^2y-\frac{y^3}{3}\right)$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
ny = 256
npy = 256
radi2 = 10
npt = ny*npy
nSample = 1024*4
nsteps = 10000
dt = 1.0e-5

In [2]:
def drift(x,px,y,py,dt):
    x = x + px*dt
    y = y + py*dt
    return x,y

def kick(x,px,y,py,dt):
    px = px - dt*(x +2*x*y)
    py = py - dt*(y +  x*x  -y*y)
    return px,py
    
def oneStep(x,px,y,py,dt):
    x,y = drift(x,px,y,py,0.5*dt)
    px,py = kick (x,px,y,py,    dt)
    x,y = drift(x,px,y,py,0.5*dt)
    return x,px,y,py

def multiStep(x,px,y,py,dt,nsteps):
    x,y = drift(x,px,y,py,0.5*dt)
    for i in range(nsteps-1):
        px,py = kick (x,px,y,py,dt)
        x,y   = drift(x,px,y,py,dt)
    px,py = kick (x,px,y,py,dt)
    x,y   = drift(x,px,y,py,0.5*dt)
    return x,px,y,py
    
# def checkLost(x,px,y,py):
#     ilive = x*x+y*y < radi2
#     x = x[ilive]
#     y = y[ilive]
#     px= px[ilive]
#     py= py[ilive]
#     return x,px,y,py    

In [3]:
def getData(energy):
    y = []
    py= []
    px= []
    i=0
    for y0 in np.linspace(-0.5,0.8,ny):
        for py0 in np.linspace(-0.5,0.5,npy):
            px2 = 2*energy - py0*py0 - y0*y0 + 2.0/3.0*y0**3
            if px2 >= 0.0:
                y.append(y0)
                py.append(py0)
                px.append(np.sqrt(px2))
                
    npt = len(y)
    print(npt)
    x =np.zeros(npt)
    y =np.array(y)
    px=np.array(px)
    py=np.array(py)
    ilive = x*x+y*y < radi2
    
    data = np.zeros([nSample,npt,4])
    data[0,:,0] = x[:]
    data[0,:,1] = px[:]
    data[0,:,2] = y[:]
    data[0,:,3] = py[:]
    for i in range(1,nSample):
        x,px,y,py = multiStep(x,px,y,py,dt,nsteps)
        ilive = np.logical_and(ilive,x*x+y*y < radi2)
        data[i,:,0] = x
        data[i,:,1] = px
        data[i,:,2] = y
        data[i,:,3] = py
    print(np.sum(ilive))
    return data[:,ilive,:]

In [ ]:
regular_data = getData(1.0/12.0)
chotic_data = getData(1.0/8.0)

27322


In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.scatter(regular_data[0,:,2],regular_data[0,:,3])
plt.scatter(regular_data[-1,:,2],regular_data[-1,:,3])
plt.subplot(1,2,2)
plt.scatter(chotic_data[0,:,2],chotic_data[0,:,3])
plt.scatter(chotic_data[-1,:,2],chotic_data[-1,:,3])

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
for i in range(64):
    plt.plot(regular_data[:,i,0])
plt.subplot(1,2,2)
for i in range(64):
    plt.plot(chotic_data[:,i,0])